INSTALL NECESSARY LIBRARIES

In [1]:
pip install gradio gdown

In [2]:
# Install PyTorch (if not already installed)
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu




# Install other required packages
!pip install opencv-python matplotlib

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
# Clone the repository
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2

# Install Detectron2
!pip install -e .

fatal: destination path 'detectron2' already exists and is not an empty directory.
/content/detectron2
Obtaining file:///content/detectron2
  Preparing metadata (setup.py) ... done
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


ACCESS GOOGLE DRIVE WHERE TRAINED MODEL IS STORED

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LOAD TRAINED MODEL AND DETECTRON2 CONFIGURATION



In [5]:
import torch
import gradio as gr
import cv2
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import Metadata
from detectron2 import model_zoo

# Set up Detectron2 model configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) # config file used to train the model
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/model_final.pth'  # Path to model weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Set threshold for inference
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12  # Set number of classes
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Set device
predictor = DefaultPredictor(cfg)

# Define class names
class_names = ['road traffic', 'bicycles', 'buses', 'crosswalks', 'fire hydrants',
               'motorcycle', 'traffic lights', 'vehicles']
my_metadata = Metadata()
my_metadata.set(thing_classes=class_names)


/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

namespace(thing_classes=['road traffic',
                         'bicycles',
                         'buses',
                         'crosswalks',
                         'fire hydrants',
                         'motorcycle',
                         'traffic lights',
                         'vehicles'])

CREATE IMAGE/VIDEO PROCESSING FUNCTION AND SET UP GRADIO INTERFACE

In [ ]:
# Function to process video frames
def predict_and_display_frame(frame):
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    outputs = predictor(im)  # Run object detection
    v = Visualizer(im[:, :, ::-1], scale=1.0, instance_mode=0, metadata=my_metadata)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    return out.get_image()[:, :, ::-1]  # Return processed frame in RGB format

def live_tracking(video_input):
    cap = cv2.VideoCapture(video_input)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = predict_and_display_frame(frame)  # Process frame
        yield processed_frame  # Yield processed frame for Gradio

def detect_objects_in_image(image):
    # Convert the image to the format required by the predictor
    im = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return predict_and_display_frame(im)



# Set up Gradio interface for live tracking
with gr.Blocks() as interface:
    gr.Markdown("# Live Object Detection with Detectron2")
    gr.Markdown("### Capture from your webcam, upload a video file, or upload an image")

    with gr.Row():
        video_input = gr.Video(label="Input Video")  # Video input
        start_button = gr.Button("Detect Objects from Uploaded Video")

    with gr.Row():
        image_input = gr.Image(label="Input Image", type="pil")  # Image input
        detect_button = gr.Button("Detect Objects in Image")

    with gr.Row():
        output_image_video = gr.Image(label="Processed Video Output", type="numpy")  # Output for video
        output_image_img = gr.Image(label="Processed Image Output", type="numpy")  # Output for image

    start_button.click(
        live_tracking,
        inputs=video_input,
        outputs=output_image_video
    )

    detect_button.click(
        detect_objects_in_image,
        inputs=image_input,
        outputs=output_image_img
    )

    gr.Markdown("### Instructions:")
    gr.Markdown("1. Choose to capture from your webcam or upload a video file.")
    gr.Markdown("2. Click the 'Start Detection' button to begin object detection on the video.")
    gr.Markdown("3. Upload an image and click 'Detect Objects in Image' to process it.")

# Launch the Gradio app
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://86d01b550f9bac651e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
